In [3]:
#exel
import openpyxl as xl
import os
#arcpy
from arcpy import *
import random
#API
import requests
import datetime
#
import json

In [60]:
#programme qui parcours les dossier et raite chacun d'eux
def extract_traffic_data(chemin_dossier):
    fichiers = [fichier for fichier in os.listdir(chemin_dossier) if os.path.isfile(os.path.join(chemin_dossier, fichier))]
    for fichier in fichiers:
        file_path=os.path.join(chemin_dossier,fichier)            
    data_tuples = []
    with open(file_path, 'r') as file:
        for line in file:
                data = json.loads(line)
                data_tuple =[(data['road_id'], data['traffic_jam_ratio'])]
                data_tuples.append(data_tuple)
    return (data_tuples)[0]

def creer_point(shp_nom, x, y, nom):
    arcpy.CreateFeatureclass_management(r"C:\Users\lenovo\Desktop\projet",shp_nom, "POINT",spatial_reference="2154")
    arcpy.AddField_management(os.path.join(r"C:\Users\lenovo\Desktop\projet",shp_nom), "Nom", "TEXT")
    with arcpy.da.InsertCursor(os.path.join(r"C:\Users\lenovo\Desktop\projet",shp_nom), ["SHAPE@XY", "Nom"]) as cursor:
        cursor.insertRow([(x, y), nom])
def lire_coordonnees_deux_points(chemin_dossier):
    nom_fichier=os.path.join(chemin_dossier,"coordonees.txt")
    with open(nom_fichier, 'r') as fichier:
        lignes = fichier.readlines()
        coordonnees_point1X = (lignes[0].split(":")[1].split(",")[0])
        coordonnees_point1Y = ( lignes[0].split(":")[1].split(",")[1])[:-1]
        coordonnees_point2X = ( lignes[0].split(":")[1].split(",")[0])
        coordonnees_point2Y = (lignes[0].split(":")[1].split(",")[1])[:-1]

        return [coordonnees_point1X, coordonnees_point1Y,coordonnees_point2X, coordonnees_point2Y]       


In [ ]:
## Modifier les données 1er cas (au cas ou on a traiter toutes les données au temps reel )
#ajouter le champ temps pour toutes les routes a l aide de leur id a partir d'un dossier contenant toutes les fichier jsons
#on suppose que toutes les fichiers json obtenu sont stockés dans un dossier a l aide d une api

#id_time list  contient des tuples exemple [(id1,taux1),(id2,taux2),....]
chemin_dossier=r"C:\Users\lenovo\Desktop\data"
layer_path=r"C:\Users\lenovo\Desktop\projet\hacka.mdb\network\paris_route"
id_time_list=extract_traffic_data(chemin_dossier)
time_field_name = 'Temps'
if not arcpy.ListFields(layer_path, time_field_name):
    arcpy.AddField_management(layer_path, time_field_name, 'DOUBLE')    
id_time_dict = dict(id_time_list)
with arcpy.da.UpdateCursor(layer_path, ['ID', time_field_name]) as cursor:
    for row in cursor:
        route_id = row[0]
        if route_id in id_time_dict:
            row[1] = id_time_dict[route_id]
        else:
            row[1] = 0  
        cursor.updateRow(row)

In [15]:
## Modifier les données 2eme ajouter les retard aleatoire (car le server ne supporte pas un grand traitement de toutes les données)
#utilise pour remplir le champs temps avec des valeur aleatoire (car le server ne supporte pas un grand traitement de toutes les données)
def add_random_time_field(shapefile_path):
    time_field_name = 'Temps_AL'
    field_names = [field.name for field in arcpy.ListFields(shapefile_path)]
    
    if time_field_name not in field_names:
        arcpy.AddField_management(shapefile_path, time_field_name, 'DOUBLE')
    
    with arcpy.da.UpdateCursor(shapefile_path, [time_field_name]) as cursor:
        for row in cursor:
            random_time = random.uniform(10, 60)
            row[0] = random_time
            cursor.updateRow(row)

In [27]:
# Remplacez par les coordonnées réelles a partir d'un fichier texte contenant les coordonéées 
#donné par l'utilisateur dans la page web
#traitement en fct de temps

X_start=lire_coordonnees_deux_points(chemin_dossier)[0]
Y_start=lire_coordonnees_deux_points(chemin_dossier)[1]
X_end=lire_coordonnees_deux_points(chemin_dossier)[2]
Y_end=lire_coordonnees_deux_points(chemin_dossier)[3]


network= r"C:\Users\lenovo\Desktop\projet\hacka.mdb\network\network_ND"
env.workspace=r"C:\Users\lenovo\Desktop\projet"

try:
    route_layer = na.MakeRouteLayer(network, "RouteOptimale", "time")
except:
    arcpy.management.Delete("RouteOptimale")
    route_layer = na.MakeRouteLayer(network, "RouteOptimale", "time")


try:
    creer_point("point_depart.shp",X_start, Y_start,"point_depart")
except:
    arcpy.management.Delete("point_depart.shp")
    creer_point("point_depart.shp",X_start, Y_start,"point_depart")
    
try:
    creer_point("point_arrive.shp",X_end, Y_end,"point_arrive")
except:
    arcpy.management.Delete("point_arrive.shp")
    creer_point("point_arrive.shp",X_end, Y_end,"point_arrive")
    
try:
    arcpy.MakeFeatureLayer_management("point_depart.shp", "point_depart_lyr")
except:
    arcpy.management.Delete("point_depart_lyr")
    arcpy.MakeFeatureLayer_management("point_depart.shp", "point_depart_lyr")
    
try:
    arcpy.MakeFeatureLayer_management("point_arrive.shp", "point_arrive_lyr")
except:
    arcpy.management.Delete("point_arrive_lyr")
    arcpy.MakeFeatureLayer_management("point_arrive.shp", "point_arrive_lyr")


# Use the feature layers in AddLocations
na.AddLocations(route_layer, "Stops", "point_depart_lyr", "", "")
na.AddLocations(route_layer, "Stops", "point_arrive_lyr", "", "")

group_layer=na.Solve(route_layer).getOutput(0)
route_sublayer=arcpy.mapping.ListLayers(group_layer, "RouteOptimale")[0]
route_optimale=arcpy.mapping.ListLayers(route_sublayer,"Routes")[0]
try:
    arcpy.management.CopyFeatures(route_optimale, "Itineraire_Optimal.shp")
except:
    arcpy.management.Delete(chemin_path)
    arcpy.management.CopyFeatures(route_optimale, "Itineraire_Optimal.shp")


Itinéraire optimal créé avec succès.


In [ ]:
import geopandas as gpd # a installer 
# ecrire le resultat "json" dans un fichier texte(insupportable par python2.0 mais il faut le traiter sur python3 pyzo par exemple)
texte_path=os.path.join(chemin_dossier,"chemin_optimal.txt")
shapefile_path ="Itineraire_Optimal.shp"

gdf = gpd.read_file(shapefile_path)

geojson_result = gdf.to_json()
with open(texte_path, 'a') as fichier:
    fichier.write(geojson_result)
